# Keyword Extraction - Data Collection

This code is the data collection part of a keyword extraction experiment. 

The code collects news articles from the german news website golem.de. As an input, it uses an excel file with links to the archive pages where all platform articles are linked in a monthly structure. The code then uses the Selenium web driver to access the page and navigate the articles. It then extracts the article text, headline, SEO meta keywords, author, time, and SEO meta description. The results are saved as excel files.

Importing all necessary packages

In [1]:
import time
import numpy as np
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor
from IPython.display import clear_output

import selenium
from selenium import webdriver
from selenium.webdriver import Firefox
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options

from webdriver_manager.firefox import GeckoDriverManager

This function gets the archive page as an input. It then navigates to every article on the archive page and extracts the text, headline, date, author, description, and SEO meta keywords.

In [45]:
def get_articles(url, driver, filename):
    
    # open url in webdriver
    driver.get(url)
    time.sleep(3)
    id_ = "https://www.golem.de/0"  + str(filename)
    
    # get a list of all article links on the page
    try:
        all_links = WebDriverWait(driver, 3).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href*="%s"]' % id_)))
    except:
        print('error', id_, driver)

    title_text_list = []
    keywords_text_list = []
    description_text_list = []
    content_text_list = []
    dates_text_list = []
    link_list = []

    j = 0
    time.sleep(3)
    # loop over the list with all article links, cklick on them, collect the data and return to the archive page
    for index, a in enumerate(all_links):

        # Check if we are again on our archive page
        try:
            if driver.current_url != url:
                time.sleep(5)
                driver.get(url)
        except:
            continue

        # Get all links from the archive page
        try:
            links = WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'a[href*="%s"]' % id)))
        except:
            print('WebDriverException: Message: target frame detached')
            j = j + 1
            print(j)
            continue

        time.sleep(0.5)

        # Click on element
        try: 
            links[j].click()
        except:
            print('Can not click on article link')
            j = j + 1
            print(j)
            continue

        # get TITLE
        try:
            title_text_list.append(driver.find_element(by = By.XPATH, value = "//meta[@name='title']").get_attribute("content"))
        except NoSuchElementException:
            print('error - no title found')
            title_text_list.append(None)

        # get LINK
        try:
            link_list.append(driver.current_url)
        except NoSuchElementException:
            print('error - no url found')
            link_list.append(None)

        # get DATE
        try:
            dates_text_list.append(driver.find_element(by = By.XPATH, value = "//*[@id='screen']/div[2]/article/header/div[1]/time").text)
        except NoSuchElementException:
            print('error - no date found')
            dates_text_list.append(None)

        # get Seo meta KEYWORDS
        try:
            keywords_text_list.append(driver.find_element(by = By.XPATH, value = "//meta[@name='news_keywords']").get_attribute("content"))
        except NoSuchElementException:
            print('error - no keywords found')
            keywords_text_list.append(None)

        # get DESCRIPTION
        try:
            description_text_list.append(driver.find_element(by = By.XPATH, value = "//meta[@name='description']").get_attribute("content"))
        except NoSuchElementException:
            print('error - no description found')
            description_text_list.append(None)

        # get article TEXT
        try:
            texts_list = []
            texts = driver.find_elements(by = By.XPATH, value = "//*[starts-with(@id, 'gpar')]")
            for element in texts:
                texts_list.append(element.text)
            content_text_list.append(' '.join(texts_list))
        except NoSuchElementException:
            print('error - no text found')
            content_text_list.append(None)

        # Go back to archive list    
        try:
            driver.execute_script("window.history.go(-1)")
        except:
            print('click back exception')
            j = j + 1
            print(j)
            continue

        time.sleep(0.5)
        j = j + 1
        print('Scraping Acticle Nr.' + str(j) + ' out of ' + str(len(all_links)))
            

    # Create dataframe with extracted data
    golem_articles = pd.DataFrame(
        {'title': title_text_list,
         'keywords': keywords_text_list,
         'description': description_text_list,
         'text': content_text_list,
         'link': link_list,
         'date': dates_text_list})
    
    return golem_articles

This function starts the selenium web driver, which is based on the Firefox web browser. It also installs an AD-blocker to avoid unnecessary waiting times due to large banner ads and speeds up the code. 
Once the website has loaded, the function clicks on the cookie accept banner.

In [18]:
def start_driver(url = 'https://www.golem.de/aa-1001.html'):
    
    # Quit driver if it is still running
    try:
        driver.quit()
    except:
        pass
    
    # Set options for webdriver
    options = Options()
    options.set_preference('permissions.default.image', 2)
    options.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-application-cache')
    options.add_argument('--disable-gpu')
    options.add_argument('--disable-dev-shm-usage')
    
    # Install Firefox webdriver
    driver = webdriver.Firefox(executable_path = GeckoDriverManager().install(), options = options) # , options=opts
    time.sleep(2)
    
    # using webdriver's install_addon API to install the downloaded Firefox extension (AD blocker)
    extension_path = "ghostery_privacy_ad_blocker-8.5.5-an+fx.xpi"
    driver.install_addon(extension_path, temporary=True)
    time.sleep(2)
    
    # Switches from tab with AD blocker to target website
    driver.switch_to.window(driver.window_handles[0])
    
    driver.get(url)
    driver.implicitly_wait(5)
    time.sleep(4)

    # Click on the cookie accept button
    button = driver.find_element(By.XPATH, "//*[starts-with(@id, 'sp_message_container_')]")
    button.click()
    time.sleep(4)
    
    return driver

This function loops over the excel file with the links to the website's archive pages and calls the get_articles function to extract the data. Then it saves the extracted data as excel files.

In [9]:
def run_scraper(df):

    # Loop over list with archive page links
    for index, row in df.iterrows():
        
        # Start the webdriver by calling start_driver()
        driver = start_driver()
        
        # Call get_articles() function
        articles_df = get_articles(row['url'], driver, row['file_name'])

        # Save the extracted data as excel file
        path = 'golem_articles/golem_archive_2_' + str(row['file_name']) + '.xlsx'
        articles_df.to_excel(path)
        
        # Quit webdriver session
        driver.quit()

This function uses multithreading via the ThreadPoolExecutor to run multiple scrapers in parallel. Multithreading tremendously speeds up the data collection process.

In [10]:
def setup_workers(workers = 2):
    
    # Read excel file with links to archive pages
    golem_archive_links = pd.read_excel('archive_links.xlsx')

    # Split up the list for multithreading
    files = np.array_split(golem_archive_links, workers)
    
    # Using multithreading to exectute the scraper multiple times
    with ThreadPoolExecutor(max_workers = workers) as executor:
        executor.map(run_scraper, files)

This cell calls the setup_workers function and defines how many workers (how many scrapers in parallel) should be launched.

In [46]:
url = 'https://www.golem.de/aa-2203.html'
setup_workers(4)





====== WebDriver manager ======


====== WebDriver manager ======


====== WebDriver manager ======
====== WebDriver manager ======
Current firefox version is 100.0
Current firefox version is 100.0
Current firefox version is 100.0
Current firefox version is 100.0
Get LATEST geckodriver version for 100.0 firefox
Get LATEST geckodriver version for 100.0 firefox
Get LATEST geckodriver version for 100.0 firefox
Get LATEST geckodriver version for 100.0 firefox
Driver [/Users/jannikmaierhoefer/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache
/var/folders/ym/8dbl8tpd2m77kbdjkly4hsx40000gn/T/ipykernel_94201/2833473687.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path = GeckoDriverManager().install(), options = options) # , options=opts
Driver [/Users/jannikmaierhoefer/.wdm/drivers/geckodriver/macos/v0.31.0/geckodriver] found in cache
Driver [/Users/jannikmaierhoefer/.wdm/drivers/gec

go to right website
go to right website
go to right website
go to right website
getting url
getting url
getting url
getting url
creating id
creating id
creating id
creating id
Scraping Acticle Nr.1 out of 537
Scraping Acticle Nr.1 out of 706
Scraping Acticle Nr.1 out of 682
Scraping Acticle Nr.1 out of 619
Scraping Acticle Nr.2 out of 706
Scraping Acticle Nr.2 out of 537
Scraping Acticle Nr.2 out of 682
Scraping Acticle Nr.2 out of 619
Scraping Acticle Nr.3 out of 537
Scraping Acticle Nr.3 out of 706
Scraping Acticle Nr.3 out of 619
Scraping Acticle Nr.3 out of 682
Scraping Acticle Nr.4 out of 537
Scraping Acticle Nr.4 out of 706
Scraping Acticle Nr.4 out of 682
Scraping Acticle Nr.4 out of 619
Scraping Acticle Nr.5 out of 537
Scraping Acticle Nr.5 out of 706
Scraping Acticle Nr.5 out of 619
Scraping Acticle Nr.5 out of 682
Scraping Acticle Nr.6 out of 537
Scraping Acticle Nr.6 out of 706
Scraping Acticle Nr.6 out of 619
Scraping Acticle Nr.6 out of 682
Scraping Acticle Nr.7 out of 537

Scraping Acticle Nr.58 out of 619
Scraping Acticle Nr.62 out of 537
Scraping Acticle Nr.61 out of 706
Scraping Acticle Nr.60 out of 682
Scraping Acticle Nr.59 out of 619
Scraping Acticle Nr.63 out of 537
Scraping Acticle Nr.62 out of 706
Scraping Acticle Nr.61 out of 682
Scraping Acticle Nr.60 out of 619
Scraping Acticle Nr.64 out of 537
Scraping Acticle Nr.63 out of 706
Scraping Acticle Nr.62 out of 682
Scraping Acticle Nr.61 out of 619
Scraping Acticle Nr.65 out of 537
Scraping Acticle Nr.64 out of 706
Scraping Acticle Nr.63 out of 682
Scraping Acticle Nr.62 out of 619
Scraping Acticle Nr.66 out of 537
Scraping Acticle Nr.65 out of 706
Scraping Acticle Nr.64 out of 682
Scraping Acticle Nr.63 out of 619
Scraping Acticle Nr.67 out of 537
Scraping Acticle Nr.65 out of 682
Scraping Acticle Nr.66 out of 706
Scraping Acticle Nr.64 out of 619
Scraping Acticle Nr.68 out of 537
Scraping Acticle Nr.67 out of 706
Scraping Acticle Nr.66 out of 682
Scraping Acticle Nr.65 out of 619
Scraping Actic

Scraping Acticle Nr.118 out of 619
Scraping Acticle Nr.116 out of 537
Scraping Acticle Nr.123 out of 682
Scraping Acticle Nr.119 out of 619
Scraping Acticle Nr.117 out of 537
error - no title found
Scraping Acticle Nr.124 out of 682
Scraping Acticle Nr.120 out of 619
Scraping Acticle Nr.118 out of 537
Scraping Acticle Nr.125 out of 682
Scraping Acticle Nr.121 out of 619
Scraping Acticle Nr.119 out of 537
Scraping Acticle Nr.126 out of 682
error - no date found
Scraping Acticle Nr.122 out of 619
Scraping Acticle Nr.120 out of 537
Scraping Acticle Nr.127 out of 682
Scraping Acticle Nr.123 out of 619
Scraping Acticle Nr.121 out of 537
Scraping Acticle Nr.128 out of 682
Scraping Acticle Nr.124 out of 619
error - no keywords found
Scraping Acticle Nr.122 out of 537
Scraping Acticle Nr.129 out of 682
Scraping Acticle Nr.125 out of 619
Scraping Acticle Nr.123 out of 537
Scraping Acticle Nr.130 out of 682
Scraping Acticle Nr.126 out of 619
Scraping Acticle Nr.124 out of 537
Scraping Acticle Nr

Scraping Acticle Nr.171 out of 706
Scraping Acticle Nr.179 out of 619
Scraping Acticle Nr.183 out of 682
Scraping Acticle Nr.178 out of 537
Scraping Acticle Nr.172 out of 706
Scraping Acticle Nr.180 out of 619
Scraping Acticle Nr.184 out of 682
Scraping Acticle Nr.179 out of 537
Scraping Acticle Nr.173 out of 706
Scraping Acticle Nr.181 out of 619
Scraping Acticle Nr.185 out of 682
Scraping Acticle Nr.180 out of 537
Scraping Acticle Nr.174 out of 706
Scraping Acticle Nr.182 out of 619
Scraping Acticle Nr.186 out of 682
Scraping Acticle Nr.181 out of 537
Scraping Acticle Nr.175 out of 706
Scraping Acticle Nr.183 out of 619
Scraping Acticle Nr.187 out of 682
Scraping Acticle Nr.182 out of 537
Scraping Acticle Nr.176 out of 706
Scraping Acticle Nr.184 out of 619
Scraping Acticle Nr.188 out of 682
Scraping Acticle Nr.183 out of 537
Scraping Acticle Nr.177 out of 706
Scraping Acticle Nr.185 out of 619
Scraping Acticle Nr.189 out of 682
Scraping Acticle Nr.184 out of 537
Scraping Acticle Nr.

...

Scraping Acticle Nr.295 out of 641
Scraping Acticle Nr.296 out of 641
Scraping Acticle Nr.297 out of 641
Scraping Acticle Nr.298 out of 641
Scraping Acticle Nr.299 out of 641
Scraping Acticle Nr.300 out of 641
Scraping Acticle Nr.301 out of 641
Scraping Acticle Nr.302 out of 641
Scraping Acticle Nr.303 out of 641
go to right website
getting url
Scraping Acticle Nr.304 out of 641
Scraping Acticle Nr.305 out of 641
creating id
Scraping Acticle Nr.306 out of 641
Scraping Acticle Nr.307 out of 641
Scraping Acticle Nr.308 out of 641
Scraping Acticle Nr.1 out of 717
Scraping Acticle Nr.309 out of 641
Scraping Acticle Nr.2 out of 717
Scraping Acticle Nr.310 out of 641
Scraping Acticle Nr.3 out of 717
Scraping Acticle Nr.311 out of 641
Scraping Acticle Nr.4 out of 717
Scraping Acticle Nr.312 out of 641
Scraping Acticle Nr.5 out of 717
Scraping Acticle Nr.313 out of 641
Scraping Acticle Nr.6 out of 717
Scraping Acticle Nr.314 out of 641
Scraping Acticle Nr.7 out of 717
Scraping Acticle Nr.8 out

Scraping Acticle Nr.423 out of 641
Scraping Acticle Nr.107 out of 717
Scraping Acticle Nr.424 out of 641
Scraping Acticle Nr.108 out of 717
Scraping Acticle Nr.425 out of 641
Scraping Acticle Nr.109 out of 717
Scraping Acticle Nr.426 out of 641
Scraping Acticle Nr.110 out of 717
Scraping Acticle Nr.427 out of 641
Scraping Acticle Nr.111 out of 717
Scraping Acticle Nr.428 out of 641
Scraping Acticle Nr.112 out of 717
Scraping Acticle Nr.429 out of 641
Scraping Acticle Nr.113 out of 717
Scraping Acticle Nr.430 out of 641
Scraping Acticle Nr.114 out of 717
Scraping Acticle Nr.431 out of 641
Scraping Acticle Nr.115 out of 717
Scraping Acticle Nr.432 out of 641
Scraping Acticle Nr.116 out of 717
Scraping Acticle Nr.433 out of 641
Scraping Acticle Nr.117 out of 717
Scraping Acticle Nr.434 out of 641
Scraping Acticle Nr.435 out of 641
Scraping Acticle Nr.436 out of 641
Scraping Acticle Nr.437 out of 641
error - no date found
Scraping Acticle Nr.118 out of 717
Scraping Acticle Nr.438 out of 64

Scraping Acticle Nr.553 out of 641
Scraping Acticle Nr.554 out of 641
Scraping Acticle Nr.555 out of 641
error - no date found
Scraping Acticle Nr.205 out of 717
Scraping Acticle Nr.556 out of 641
Scraping Acticle Nr.206 out of 717
Scraping Acticle Nr.557 out of 641
Scraping Acticle Nr.207 out of 717
Scraping Acticle Nr.558 out of 641
Scraping Acticle Nr.208 out of 717
Scraping Acticle Nr.559 out of 641
Scraping Acticle Nr.209 out of 717
Scraping Acticle Nr.560 out of 641
Scraping Acticle Nr.210 out of 717
Scraping Acticle Nr.561 out of 641
Scraping Acticle Nr.211 out of 717
Scraping Acticle Nr.562 out of 641
Scraping Acticle Nr.563 out of 641
Scraping Acticle Nr.212 out of 717
Scraping Acticle Nr.564 out of 641
Scraping Acticle Nr.213 out of 717
Scraping Acticle Nr.565 out of 641
Scraping Acticle Nr.214 out of 717
Scraping Acticle Nr.566 out of 641
Scraping Acticle Nr.215 out of 717
Scraping Acticle Nr.216 out of 717
Scraping Acticle Nr.567 out of 641
Scraping Acticle Nr.217 out of 71

Scraping Acticle Nr.349 out of 717
Scraping Acticle Nr.350 out of 717
Scraping Acticle Nr.351 out of 717
Scraping Acticle Nr.352 out of 717
Scraping Acticle Nr.353 out of 717
Scraping Acticle Nr.354 out of 717
Scraping Acticle Nr.355 out of 717
Scraping Acticle Nr.356 out of 717
Scraping Acticle Nr.357 out of 717
Scraping Acticle Nr.358 out of 717
Scraping Acticle Nr.359 out of 717
Scraping Acticle Nr.360 out of 717
error - no date found
Scraping Acticle Nr.361 out of 717
Scraping Acticle Nr.362 out of 717
Scraping Acticle Nr.363 out of 717
Scraping Acticle Nr.364 out of 717
Scraping Acticle Nr.365 out of 717
Scraping Acticle Nr.366 out of 717
Scraping Acticle Nr.367 out of 717
Scraping Acticle Nr.368 out of 717
Scraping Acticle Nr.369 out of 717
Scraping Acticle Nr.370 out of 717
Scraping Acticle Nr.371 out of 717
Scraping Acticle Nr.372 out of 717
Scraping Acticle Nr.373 out of 717
Scraping Acticle Nr.374 out of 717
Scraping Acticle Nr.375 out of 717
Scraping Acticle Nr.376 out of 71

error - no date found
Scraping Acticle Nr.565 out of 717
error - no date found
Scraping Acticle Nr.566 out of 717
error - no date found
Scraping Acticle Nr.567 out of 717
error - no date found
Scraping Acticle Nr.568 out of 717
error - no date found
Scraping Acticle Nr.569 out of 717
Scraping Acticle Nr.570 out of 717
Scraping Acticle Nr.571 out of 717
Scraping Acticle Nr.572 out of 717
Scraping Acticle Nr.573 out of 717
Scraping Acticle Nr.574 out of 717
Scraping Acticle Nr.575 out of 717
Scraping Acticle Nr.576 out of 717
Scraping Acticle Nr.577 out of 717
Scraping Acticle Nr.578 out of 717
error - no date found
Scraping Acticle Nr.579 out of 717
Scraping Acticle Nr.580 out of 717
Scraping Acticle Nr.581 out of 717
Scraping Acticle Nr.582 out of 717
Scraping Acticle Nr.583 out of 717
Scraping Acticle Nr.584 out of 717
Scraping Acticle Nr.585 out of 717
error - no date found
Scraping Acticle Nr.586 out of 717
error - no date found
Scraping Acticle Nr.587 out of 717
Scraping Acticle Nr

### References:

Selenium: https://www.selenium.dev/documentation/ <br>
Golem: https://www.golem.de <br>
ThreadPoolExecutor: https://docs.python.org/3/library/concurrent.futures.html <br>
Ad blocker: https://addons.mozilla.org/en-US/firefox/addon/ghostery/versions/ <br>
